In [3]:
import pandas as pd
# orig = pd.read_pickle("/Users/gil/Downloads/processed_data_yield_spread_with_similar_trades_v2_original.pkl")
# new = pd.read_pickle("/Users/gil/Downloads/2025-06-18_2025-01-01_trades_for_all_dates_from_get_processed_data.pkl")

# orig.head()

# new.head()

# orig.columns

# new.columns


In [4]:
# df_orig = orig[orig.trade_date == "2025-05-01"]
# df_new = new[new.trade_date == "2025-05-01"]

# df_orig.head()

df_new = pd.read_csv("2025-05-01_new.csv")
df_orig = pd.read_csv("2025-05-01_orig.csv")

df_new.head()

,Unnamed: 0,rtrs_control_number,cusip,yield,is_callable,refund_date,accrual_date,dated_date,next_sink_date,coupon,...,D_min_ago_ago,D_min_ago_qdiff,P_min_ago_ys,P_min_ago_ttypes,P_min_ago_ago,P_min_ago_qdiff,S_min_ago_ys,S_min_ago_ttypes,S_min_ago_ago,S_min_ago_qdiff
0,2063959,2025050123093100,544532JX3,370.4,False,NaN,2024-04-02,2024-04-02,NaN,5.00,...,4.248660,3.699057,80.052697,PS,4.063559,4.954247,63.252697,SS,4.248660,3.699057
1,2063960,2025050123093200,544532JX3,382.3,False,NaN,2024-04-02,2024-04-02,NaN,5.00,...,4.248660,3.699057,80.052697,PD,4.063559,4.954247,63.252697,SD,4.248660,3.699057
2,2063961,2025050123092600,7087966Z5,375.9,True,NaN,2017-09-28,2017-09-28,NaN,2.55,...,5.732462,0.000000,136.344097,PS,5.732462,0.000000,61.306449,SS,6.645146,4.176120
3,2063962,2025050123092700,7087966Z5,387.0,True,NaN,2017-09-28,2017-09-28,NaN,2.55,...,5.732462,0.000000,136.344097,PD,5.732462,0.000000,61.306449,SD,6.645146,4.176120
4,2063963,2025050123077000,576002BK4,358.2,False,NaN,2005-07-27,2005-07-27,NaN,5.50,...,5.052559,3.699057,59.724004,PD,5.052559,3.699057,-36.121328,SD,6.091165,0.000000


In [5]:
import pandas as pd
import numpy as np

# Merge the dataframes on rtrs_control_number
merged = pd.merge(
    df_orig, 
    df_new, 
    on='rtrs_control_number', 
    suffixes=('_orig', '_new'),
    how='outer',
    indicator=True
)

# Check merge results
print("Merge results:")
print(merged['_merge'].value_counts())
print("\n")

# Find rows that exist in both
both = merged[merged['_merge'] == 'both'].copy()
print(f"Number of matching rtrs_control_numbers: {len(both)}")

# Helper function to safely compare values
def safe_compare(val1, val2):
    """Compare two values, handling arrays, NaN, etc."""
    # Both are NaN/None
    if pd.isna(val1) and pd.isna(val2):
        return True
    # One is NaN, other isn't
    if pd.isna(val1) or pd.isna(val2):
        return False
    # Handle arrays/lists
    if isinstance(val1, (list, np.ndarray)) or isinstance(val2, (list, np.ndarray)):
        try:
            return np.array_equal(val1, val2)
        except:
            return str(val1) == str(val2)
    # Regular comparison
    try:
        return val1 == val2
    except:
        return str(val1) == str(val2)

# Function to compare columns
def compare_columns(df, orig_suffix='_orig', new_suffix='_new'):
    # Get base column names (without suffixes)
    base_cols = [col.replace(orig_suffix, '').replace(new_suffix, '') 
                 for col in df.columns 
                 if col.endswith(orig_suffix)]
    
    differences = {}
    
    for col in base_cols:
        orig_col = f"{col}{orig_suffix}"
        new_col = f"{col}{new_suffix}"
        
        if orig_col in df.columns and new_col in df.columns:
            # Count differences row by row
            diff_count = 0
            for idx, row in df.iterrows():
                if not safe_compare(row[orig_col], row[new_col]):
                    diff_count += 1
            
            if diff_count > 0:
                differences[col] = {
                    'count': diff_count,
                    'percentage': (diff_count / len(df)) * 100
                }
    
    return differences

# Get differences
print("Analyzing differences (this may take a moment)...")
diffs = compare_columns(both)

# Sort by number of differences
sorted_diffs = sorted(diffs.items(), key=lambda x: x[1]['count'], reverse=True)

print("\nColumns with differences (sorted by count):")
print("-" * 60)
for col, info in sorted_diffs[:20]:  # Show top 20
    print(f"{col:<30} {info['count']:>6} differences ({info['percentage']:>5.1f}%)")

# Show some anecdotal examples
print("\n\nAnecdotal examples of differences:")
print("=" * 80)

# Pick a few interesting columns to show examples
example_cols = ['yield', 'dollar_price', 'last_yield_spread', 'last_seconds_ago', 
                'trade_history', 'last_trade_type', 'last_size']

for col in example_cols:
    if col in [c for c, _ in sorted_diffs]:
        print(f"\n{col.upper()} differences:")
        print("-" * 40)
        
        orig_col = f"{col}_orig"
        new_col = f"{col}_new"
        
        # Find rows with differences
        diff_indices = []
        for idx, row in both.iterrows():
            if not safe_compare(row[orig_col], row[new_col]):
                diff_indices.append(idx)
                if len(diff_indices) >= 5:  # Only show 5 examples
                    break
        
        for idx in diff_indices:
            row = both.loc[idx]
            rtrs = row['rtrs_control_number']
            orig_val = row[orig_col]
            new_val = row[new_col]
            cusip = row.get('cusip_orig', row.get('cusip_new', 'N/A'))
            
            print(f"RTRS: {rtrs}, CUSIP: {cusip}")
            
            # Special formatting for arrays
            if isinstance(orig_val, (list, np.ndarray)):
                print(f"  Original: array with shape {np.array(orig_val).shape}")
                print(f"  New:      array with shape {np.array(new_val).shape}")
                # Show first few elements if they're arrays
                try:
                    orig_arr = np.array(orig_val)
                    new_arr = np.array(new_val)
                    if orig_arr.size > 0 and new_arr.size > 0:
                        print(f"    Original first element: {orig_arr.flat[0]}")
                        print(f"    New first element: {new_arr.flat[0]}")
                except:
                    pass
            else:
                print(f"  Original: {orig_val}")
                print(f"  New:      {new_val}")
            print()

# Count total differences per row
print("\n\nAnalyzing differences per row...")
base_cols = [col.replace('_orig', '').replace('_new', '') 
             for col in both.columns 
             if col.endswith('_orig')]

diff_counts_per_row = []
for idx, row in both.iterrows():
    diff_count = 0
    for col in base_cols:
        if col != 'rtrs_control_number':
            orig_col = f"{col}_orig"
            new_col = f"{col}_new"
            if orig_col in both.columns and new_col in both.columns:
                if not safe_compare(row[orig_col], row[new_col]):
                    diff_count += 1
    diff_counts_per_row.append(diff_count)

# Calculate statistics
num_identical = sum(1 for count in diff_counts_per_row if count == 0)
print(f"\nCompletely identical rows: {num_identical} out of {len(both)} ({(num_identical/len(both))*100:.1f}%)")
print(f"Rows with at least one difference: {len(both) - num_identical}")

# Show distribution
print("\n\nDistribution of differences per row:")
diff_series = pd.Series(diff_counts_per_row)
print(diff_series.value_counts().sort_index().head(10))

/var/folders/6s/gj2fjqpx7kd1zydwmjp477dw0000gn/T/ipykernel_49140/458713983.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged = pd.merge(


Merge results:
both          71458
right_only     2818
left_only       677
Name: _merge, dtype: int64


Number of matching rtrs_control_numbers: 71458
Analyzing differences (this may take a moment)...

Columns with differences (sorted by count):
------------------------------------------------------------
Unnamed: 0                      71458 differences (100.0%)
similar_trade_history           51213 differences ( 71.7%)
issue_amount                    39086 differences ( 54.7%)
P_min_ago_qdiff                 33722 differences ( 47.2%)
S_min_ago_qdiff                 32699 differences ( 45.8%)
D_min_ago_qdiff                 32551 differences ( 45.6%)
min_ys_qdiff                    31477 differences ( 44.0%)
max_ys_qdiff                    31407 differences ( 44.0%)
max_qty_qdiff                   31275 differences ( 43.8%)
min_ago_qdiff                   31200 differences ( 43.7%)
quantity                        28332 differences ( 39.6%)
target_attention_features       28332 differ